# Анализ книжного сервиса

**Задача проекта** - проанализировать базу данных сервиса для чтения книг по подписке. В базе содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

В рамках проекта выполним 5 заданий:
- Посчитаем, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитаем количество обзоров и среднюю оценку;
- Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — таким образом исключим из анализа брошюры;
- Определим автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками;
- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Обзор данных 

Подключимся к базе данных и загрузим таблицы. Выведем первые строки таблиц для просмотра.

In [36]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                        db_config['pwd'], 
                                                        db_config['host'], 
                                                        db_config['port'], 
                                                        db_config['db']) 

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Изучим таблицы. Выведем первые 5 строк. Для этого воспользумся циклом.

In [38]:
for i in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    print('Таблица ', i)
    display(pd.io.sql.read_sql(f'''SELECT * FROM {i} LIMIT 5''', con = engine))
    print()
    print()

Таблица  books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268




Таблица  authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd




Таблица  publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company




Таблица  ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2




Таблица  reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Описание данных**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Выполнение заданий

Напишем функцию для просмотра результата запросов.

In [ ]:
def sql_result(query):
    result = pd.io.sql.read_sql(query, con = engine)
    return result

### Сколько книг вышло после 1 января 2000 года

In [45]:
books_count = ''' 
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
             ''' 
sql_result(books_count)


,count
0,819


**Вывод**

В базе найдено 819 книг, выпущенные после 1 января 2000 года. 

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [44]:
rating_review = '''
SELECT b.book_id,
       b.title,
       avg_rating,
       total_reviews
FROM books AS b
LEFT JOIN
  (SELECT book_id,
          AVG(rating) AS avg_rating
   FROM ratings
   GROUP BY book_id) AS ra ON b.book_id=ra.book_id
LEFT JOIN
  (SELECT book_id,
          COUNT(review_id) AS total_reviews
   FROM reviews
   GROUP BY book_id) AS re ON b.book_id=re.book_id
ORDER BY total_reviews DESC
LIMIT 10; 
'''

sql_result(rating_review)

,book_id,title,avg_rating,total_reviews
0,191,Disney's Beauty and the Beast (A Little Golden...,4.000000,NaN
1,387,Leonardo's Notebooks,4.000000,NaN
2,83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667,NaN
3,808,The Natural Way to Draw,3.000000,NaN
4,221,Essential Tales and Poems,4.000000,NaN
5,672,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000,NaN
6,948,Twilight (Twilight #1),3.662500,7.0
7,497,Outlander (Outlander #1),4.125000,6.0
8,750,The Hobbit or There and Back Again,4.125000,6.0
9,854,The Road,3.772727,6.0


**Выводы**

Мы получили таблицу с данными о рейтинге и количестве обзоров для каждой книги. Данные отсортированы в порядке убывания по рейтингу.  

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [43]:
top_publisher = ''' 
SELECT publisher
FROM publishers
WHERE publisher_id IN
    (SELECT publisher_id
    FROM books
    WHERE num_pages > 50
    GROUP BY publisher_id
    ORDER BY COUNT(book_id) DESC
    LIMIT 5);
'''

sql_result(top_publisher)


,publisher
0,Penguin Books
1,Vintage
2,Grand Central Publishing
3,Penguin Classics
4,Bantam


**Выводы**

В таблице выведено топ-5 издательств, которые выпускали книги толще 50 страниц. Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books. 


### Определим автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками

In [42]:
top_author = ''' 
SELECT author,
      AVG(avg_rating) AS avg_rating,
      SUM(num_of_ratings) AS num_of_ratings
FROM authors AS a
INNER JOIN books AS b ON a.author_id=b.author_id
INNER JOIN
  (SELECT book_id,
          AVG(rating) AS avg_rating,
          COUNT(rating_id) AS num_of_ratings
  FROM ratings
  GROUP BY book_id
  HAVING COUNT(rating_id) >= 50
  ORDER BY AVG(rating) DESC) AS r ON b.book_id=r.book_id
GROUP BY author
ORDER BY AVG(avg_rating) DESC
LIMIT 5;
'''

sql_result(top_author)


,author,avg_rating,num_of_ratings
0,J.K. Rowling/Mary GrandPré,4.283844,310.0
1,Markus Zusak/Cao Xuân Việt Khương,4.264151,53.0
2,J.R.R. Tolkien,4.258446,162.0
3,Louisa May Alcott,4.192308,52.0
4,Rick Riordan,4.080645,62.0


**Выводы**

В таблице выведены топ-5 самых высоко оцениваемых авторов. Автор с самой высокой средней оценкой книг(4.3) - J.K. Rowling/Mary GrandPré. 

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [41]:
avg_reviews = ''' 
WITH TEMP AS
  (SELECT username,
          COUNT(review_id) AS number_of_reviews
  FROM reviews
  WHERE username IN
      (SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id) > 50)
  GROUP BY username)
SELECT AVG(number_of_reviews)
FROM TEMP;
'''

sql_result(avg_reviews)


,avg
0,24.333333


**Выводы**

В среднем активные пользователи, которые поставили больше 50 оценок, написали по 24 обзора.

**По итогам запросов для данных из предоставленной БД:**

* После 1 января 2000 года было выпущенно 819 книг.
* Больше всего обзоров (7) у книги "Сумерки" , средний рейтинг книги - 3.7.
* Издательство, которое выпустило наибольшее число книг (42) толще 50 страниц - Penguin Books.
* Автор с самой высокой средней оценкой книг(4.3) - Джоан Роулинг.
* В среднем пользователи, которые поставил больше 50 оценок, написали 24 обзора.


